# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('ec_dwt.npz') 

# Access the concatenated features from the .npz file
X_dwt = data['X_dwt']

# Close the loaded file
data.close()

#print the merged array
X_dwt

array([[[-1.90864583e-06, -1.01319508e-06, -1.25947210e-06, ...,
         -2.17607348e-07, -3.60696991e-07,  4.04866575e-07],
        [-4.16825421e-05, -4.20260902e-05, -4.24717352e-05, ...,
         -2.03555431e-07, -3.33857556e-07,  3.57200206e-07],
        [-1.90472102e-05, -1.92407524e-05, -1.70617167e-05, ...,
         -4.33967060e-08, -5.20276540e-08, -3.12188367e-08],
        ...,
        [ 6.94002021e-06,  6.58408463e-06,  6.57031824e-06, ...,
          8.69173749e-08, -1.77016500e-08,  3.94567816e-07],
        [ 2.41499888e-05,  2.47479670e-05,  2.44680713e-05, ...,
          1.14848042e-07,  1.76879477e-07, -1.72898476e-07],
        [-1.60015896e-06, -3.41169756e-06, -2.49139569e-06, ...,
          3.27513445e-07,  7.51374528e-07, -1.43600588e-06]],

       [[ 4.44703238e-05,  4.35599754e-05,  4.57796671e-05, ...,
         -8.22744389e-08, -2.28104073e-07,  4.71143282e-07],
        [ 5.61311547e-06,  4.87111596e-06,  6.55256693e-06, ...,
          1.00866868e-08,  1.83770888e

In [3]:
data = np.load('eyes_close_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('eyes_close_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [7]:
# Check the shape of the concatenated arrays
print(X_dwt.shape, Y.shape, groups_array.shape)

(17325, 20, 205) (17325,) (17325,)


In [8]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [9]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [11]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_dwt, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_dwt[train_index], X_dwt[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
61/61 [==============================] - 16s 59ms/step - loss: 0.4291 - accuracy: 0.8016 - val_loss: 0.3839 - val_accuracy: 0.8644
Epoch 2/200
61/61 [==============================] - 2s 30ms/step - loss: 0.2577 - accuracy: 0.9001 - val_loss: 0.2672 - val_accuracy: 0.9083
Epoch 3/200
61/61 [==============================] - 2s 30ms/step - loss: 0.2215 - accuracy: 0.9137 - val_loss: 0.2532 - val_accuracy: 0.9117
Epoch 4/200
61/61 [==============================] - 2s 27ms/step - loss: 0.1957 - accuracy: 0.9211 - val_loss: 0.2571 - val_accuracy: 0.9054
Epoch 5/200
61/61 [==============================] - 2s 28ms/step - loss: 0.1798 - accuracy: 0.9288 - val_loss: 0.2628 - val_accuracy: 0.8990
Epoch 6/200
61/61 [==============================] - 2s 29ms/step - loss: 0.1666 - accuracy: 0.9316 - val_loss: 0.2678 - val_accuracy: 0.9008
Epoch 7/200
61/61 [==============================] - 2s 28ms/step - loss: 0.1523 - accuracy: 0.9378 - val_loss: 0.2578 - val_accuracy: 0.895

In [12]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [13]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9365,0.9157,0.9636,0.9390,0.9361,0.9087,0.9636,0.8729,0.8742
1,0.9469,0.9338,0.9636,0.9485,0.9467,0.9297,0.9636,0.8938,0.8942
2,0.9475,0.9447,0.9522,0.9484,0.9474,0.9426,0.9522,0.8949,0.8950
3,0.9348,0.9256,0.9477,0.9365,0.9346,0.9215,0.9477,0.8695,0.8698
4,0.9325,0.9168,0.9534,0.9347,0.9322,0.9110,0.9534,0.8649,0.8656
5,0.9463,0.9456,0.9488,0.9472,0.9463,0.9437,0.9488,0.8926,0.8926
6,0.9313,0.9077,0.9625,0.9343,0.9308,0.8992,0.9625,0.8624,0.8641
7,0.9417,0.9313,0.9556,0.9433,0.9415,0.9273,0.9556,0.8833,0.8836
8,0.9353,0.9191,0.9568,0.9376,0.9350,0.9132,0.9568,0.8706,0.8713
9,0.9480,0.9438,0.9545,0.9491,0.9479,0.9414,0.9545,0.8960,0.8961


In [14]:
metrics_df.round(4).to_csv('DWT_EC_CNN_LSTM.csv', index = False)

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
result = pd.read_csv("DWT_EC_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9365,0.9157,0.9636,0.9390,0.9361,0.9087,0.9636,0.8729,0.8742
1,0.9469,0.9338,0.9636,0.9485,0.9467,0.9297,0.9636,0.8938,0.8942
2,0.9475,0.9447,0.9522,0.9484,0.9474,0.9426,0.9522,0.8949,0.8950
3,0.9348,0.9256,0.9477,0.9365,0.9346,0.9215,0.9477,0.8695,0.8698
4,0.9325,0.9168,0.9534,0.9347,0.9322,0.9110,0.9534,0.8649,0.8656
5,0.9463,0.9456,0.9488,0.9472,0.9463,0.9437,0.9488,0.8926,0.8926
6,0.9313,0.9077,0.9625,0.9343,0.9308,0.8992,0.9625,0.8624,0.8641
7,0.9417,0.9313,0.9556,0.9433,0.9415,0.9273,0.9556,0.8833,0.8836
8,0.9353,0.9191,0.9568,0.9376,0.9350,0.9132,0.9568,0.8706,0.8713
9,0.9480,0.9438,0.9545,0.9491,0.9479,0.9414,0.9545,0.8960,0.8961


In [17]:
(result.mean()*100).round(2)

Accuracy       94.01
Precision      92.84
Recall         95.59
F1             94.19
Sensitivity    93.99
Specificity    92.38
ROC_AUC        95.59
Kappa          88.01
MCC            88.06
dtype: float64

In [18]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])